# CIFAR-10 dataset

## Save the model directory in an env variable

In [1]:
import os

#change the directory is needed
os.environ['CIFAR10_DIR']='logdir-big/1527108431'

## Perform operations on the graph to obtain the quantized model

### Freeze the graph

In [2]:
!freeze_graph --input_saved_model_dir=$CIFAR10_DIR --output_graph=$CIFAR10_DIR/frozen.pb \
    --output_node_names=final_result

Converted 16 variables to const ops.


### Optimize the graph

In [3]:
!python -m tensorflow.python.tools.optimize_for_inference --input=$CIFAR10_DIR/frozen.pb \
    --output=$CIFAR10_DIR/optimized.pb --input_names="input" --output_names="final_result"

### Add the optimized graph to Tensorboard

In [4]:
!python -m scripts.graph_pb2tb $CIFAR10_DIR $CIFAR10_DIR/optimized.pb

### Create the quantized model

In [5]:
!python -m scripts.quantize_graph --input=$CIFAR10_DIR/optimized.pb --output=$CIFAR10_DIR/eightbit.pb \
    --output_node_names=final_result --mode=eightbit

Instructions for updating:
`tf.quantize_v2` is deprecated, please use `tf.quantize` instead.


## Models size

In [6]:
!du -h $CIFAR10_DIR/optimized.pb
!du -h $CIFAR10_DIR/eightbit.pb

55M	logdir-big/1527108431/optimized.pb
14M	logdir-big/1527108431/eightbit.pb


## Accuracy, Average prediction time and cache utilization

### not-quantized model

In [7]:
!perf stat -r 10 -e cache-references,cache-misses python evaluate.py --graph $CIFAR10_DIR/optimized.pb

Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.134832128s
Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.135666193s
Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.135170157s
Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.135303965s
Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.134883049s
Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.135358184s
Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.135794709s
Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.135445475s
Evaluating model with 10000 images
               Accuracy: 0.663900000
Average prediction time: 0.135532253s
Evaluating

### quantized model

In [8]:
!perf stat -r 10 -e cache-references,cache-misses python evaluate.py --graph $CIFAR10_DIR/eightbit.pb

Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.056849577s
Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.056731763s
Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.056886843s
Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.057023433s
Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.056960484s
Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.056888494s
Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.056922880s
Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.056748064s
Evaluating model with 10000 images
               Accuracy: 0.657700000
Average prediction time: 0.056997140s
Evaluating